# Downloads all the stocks to local location

In [25]:
#Load the dataframe 
import numpy as np
import pandas as pd
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import time
import subprocess
Base_url=("https://zerodha.com/margin-calculator/Equity/")
page=requests.get(Base_url)
soup=BeautifulSoup(page.content,'html.parser')
table = soup.find('table', attrs={'class':'data equity'})
table_body = table.find('tbody')

rows = soup.find_all('tr')
data = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

df = pd.DataFrame(data = data)

company_symbol = []
[company_symbol.append(df.loc[i][1][:-3:]) for i in range(len(df)) if df.loc[i][3] == "12.5x"]
x=company_symbol
y=['BHARATFIN','TIFIN','VIJAYABANK']
company_symbol1=[item for item in x if item not in y]
company_symbol2=[item.replace('&','%26') for item in company_symbol1]
#baseURL='https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='
#for i in company_symbol2[:]:
    #textfile = subprocess.check_output(['curl','-o',i+'.txt','-O', baseURL+i])

# Analysis of the stocks

In [26]:
from datetime import datetime

def job_function2():
    start_time = datetime.now()
    symbol_list = []
    buy_list = []
    sell_list = []
    lastP_list = []
    for i in company_symbol2[:]:
        f = open(i+".txt","r")
        file = str(f.read())
        beg = file.find('[{')
        end = file.find('}]')
        trial = file[beg+1:end+1]
        f.close()
        trial = trial.replace('false','"false"').replace('true','"true"')
        trial = eval(trial)

        symbol_list.append(trial["symbol"])
        buy_list.append(trial["totalBuyQuantity"])
        sell_list.append(trial["totalSellQuantity"])
        lastP_list.append(trial["lastPrice"])
    
    global scheduler, count

    df["symbol"] = symbol_list
    df["buy"] = buy_list
    df["sell"] = sell_list
    df["lastP"] = lastP_list
    df['sell']=[x.replace('-','0').replace(',','') for x in df['sell']]
    df['buy']=[x.replace('-','0').replace(',','') for x in df['buy']]
 
    currentDT = datetime.now()
    df['time']=currentDT.strftime("%H:%M")
    
    if count==0:
        df["bs"+str(count)] = np.where(df["sell"]>=df["buy"],"S","B")
    else:
        df["bs"+str(count)] = np.where(df["sell"]>=df["buy"],"S","B")
        result["bs"+str(count)] = np.where(df["bs"+str(count)]==df["bs"+str(count-1)],"S",df["symbol"]+df["time"])

    df.to_csv("all"+str(count)+".csv")
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    
    count+=1
    
    if count == 10:
        scheduler.shutdown(wait = False)

In [27]:
!pip install apscheduler

In [28]:
import pandas as pd
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler
count = 0

df = pd.DataFrame(columns = ["symbol","buy","sell","lastP","time"])  
result = pd.DataFrame(columns = ["symbol"])
scheduler = BackgroundScheduler()

scheduler.add_job(job_function2, 'interval', seconds = 5, start_date='2019-07-23 19:11:00', end_date='2019-07-23 19:15:00', id='my_job_id')

try:
    scheduler.start()
except apscheduler.schedulers.SchedulerNotRunningError as e:
    print('program exited')

# Removes all the files present in the directory

In [29]:
import os

# Return all files in dir, and all its subdirectories, ending in pattern
def gen_files(dir, txt):
    for dirname, subdirs, files in os.walk(dir):
        for f in files:
            if f.endswith(txt):
                yield os.path.join(dir, f)

# Remove all files in the current dir matching *.config
for f in gen_files('.', '.txt'):
    os.remove(f)

In [30]:
df

,symbol,buy,sell,lastP,time
